In [1]:
import sys
from sklearn.externals import  joblib
from sklearn.grid_search import GridSearchCV
from sklearn.svm import  LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import  TfidfVectorizer,CountVectorizer
import os
import numpy as np
import string
from keras import backend
from keras.layers import Dense, Input, Lambda, LSTM, TimeDistributed
from keras.layers.merge import concatenate
from keras.layers.embeddings import Embedding
from keras.models import Model
from gensim.models.keyedvectors import KeyedVectors
import json


c:\users\dh kim\appdata\local\programs\python\python35\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
c:\users\dh kim\appdata\local\programs\python\python35\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
Using TensorFlow backend.
c:\users\dh kim\appdata\local\programs\python\python35\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected 

In [2]:
def isNumber(s):
  try:
    float(s)
    return True
  except ValueError:
    return False

In [3]:
x_text_list = []
y_text_list = []
enc = sys.getdefaultencoding()
with open("refined_category_dataset.dat",encoding=enc) as fin:
    for line in fin.readlines():
#         print (line)
        info = json.loads(line.strip())
        x_text_list.append((info['pid'],info['name']))
        y_text_list.append(info['cate'])
y_name_id_dict = joblib.load("y_name_id_dict.dat")
y_id_list = [y_name_id_dict[x] for x in y_text_list]

In [4]:
from sklearn.model_selection import train_test_split

vectorizer = CountVectorizer()
x_list = vectorizer.fit_transform(map(lambda i : i[1],x_text_list))
y_list = [y_name_id_dict[x] for x in y_text_list]
#print(x_text_list)
tmp_list = list(map(lambda i : i[1],x_text_list))
#print(tmp_list)

In [5]:
from konlpy.corpus import kolaw
from konlpy.tag import *
from konlpy.tag import Kkma
from konlpy.utils import pprint
kkma = Kkma()
nouns_list=[]
for i in range(0,len(tmp_list)):
    new_list=kkma.nouns(tmp_list[i])
    no_num_data=[]
    tt=""
    for j in new_list:
        if isNumber(j) is False :
            for k in range(0,len(j)):
                tt+=j[k]
            tt+=' '
    nouns_list.append((x_text_list[i][0],tt))
#print (nouns_list)


[('1234621373', '추천 추천정품 정품 심플 심플베드 베드 엘레강스 고급 고급형 형 접이 접이식침대 식 침대 더블 더블사이즈115 사이즈 수동 등받이 등받이6단 단 각도 각도조절 조절 '), ('1543513007', '소형 소형제도판 제도 판 '), ('1546650266', '웨빙 포켓 포켓조끼 조끼 빅 빅사이즈 사이즈 아웃 '), ('1589443530', '텍스 전 전동접이식침대 동접 이식 침대 전동침대 동 간병 침 '), ('1520510486', '플래그 메세지 마 향초 양초 양초캔 캔 선물 아로마 아로마캔 향초캔 '), ('1512822613', '광일 광일체어 체어 스카치 형 3인 인 등 등무 무 장의자 '), ('1525985492', '광일 광일체어 체어 스파 우드 2인 인 등유 로비 장의자 '), ('1326388222', '라 라움스튜디오 움 스튜디오 스크류 블랙 블랙봉 봉 일반 일반-2.5 커튼 커튼봉 일반봉 '), ('1426607155', '모슬 모슬리메모리즈 리 메모리 즈 산토리 브리즈 중 아로마 아로마캔 캔 디퓨저 소이 소이캔 소이왁스 왁스 '), ('1334473563', '무료 무료배송 배송 양키 양키캔 캔 악세사리 윅 윅디 디 심지 심지가위 가위 라이터 '), ('1333760160', '시그마 원형 센서 센서등 등 계단등 현관 현관등 '), ('1509183201', '텍스 접이 접이식침대 식 침대 열 열매트 매트 이동식 이동식침대 기능성 기능성침대 간이 간이침대 '), ('1640548937', '테리 소품 빈티지 수납함 자 에펠 에펠탑 탑 수납 수납의자 의자 '), ('1193914117', '자동 목 목쿠션 쿠션 등산 등산용품 용품 아웃 아웃도어용품 도어 야외 야외캠핑용품 캠핑 산악 산악용품 등산장비 장비 캠핑용품 아웃도어 '), ('1368179147', '추천 코스 슬림 코스모스 모스 오픈 오픈비누 비누 빨래 빨래비누통 비누통 기타 기타욕실용품 욕실 용품 고급 고급욕실용품 주방 주방욕실용품 세수 '), 

In [6]:

svc_param = np.logspace(-1,1,4)
gsvc = GridSearchCV(LinearSVC(), param_grid= {'C': svc_param}, cv = 5, n_jobs = 4)

In [7]:
new_nouns_list=list(map(lambda i : noums_list))
print(new_nouns_list)
for i in range(0,len(nouns_list)):
    tt=""
    for j in range(0,len(nouns_list[i])):
        tt+=nouns_list[i][j]
        tt+=' '
    new_nouns_list.append(tt)
#print(new_nouns_list)


In [8]:
gsvc.fit(vectorizer.transform(new_nouns_list), y_list)

GridSearchCV(cv=5, error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'C': array([  0.1    ,   0.46416,   2.15443,  10.     ])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [9]:
print(gsvc.best_score_, gsvc.best_params_)

0.6701176470588235 {'C': 0.10000000000000001}


In [10]:
eval_x_text_list = []
with open("soma8_test_data.dat",encoding=enc) as fin:
    for line in fin.readlines():
        info = json.loads(line.strip())
        eval_x_text_list.append((info['pid'],info['name']))


In [11]:
print(eval_x_text_list)

[('720988653', 'acchousee 스프레이/기아/오피러스 /2006-2007/Z8/라이트메탈/주문제품(15일예상)'), ('1378489203', '[엑스포넷주식회사]코일탭_M162.0_18.6mm_1EA 절삭공구 작업공구 공구류 코일탭 탭 공구 빠른배송'), ('1621140511', '[오가닉맘]펭코남아목도리(MBW7MF02) (인천점)'), ('1549548735', 'TASCO INFOCUS 골프거리측정기 골프거리 측정 망원경'), ('1642215271', '(KNF)극동고성능해빙기/고성능전기해빙기/10kw'), ('1594181530', '(P)인두 플라이어 드라이버등 공구세트 수공구 공구'), ('1002465039', '[토이트론]실바니안패밀리-유치원학예회세트(3589)'), ('1592587497', '[누리]사이즈가 크고 튼튼한 뉴테크 좌식 바베큐그릴'), ('181355585', '국내산 혀클리너 x 1개(낱개)혀 크리너/입냄새 제거/구강 청결제/여행용/혀간 칫솔 세트/치실/치약/백태'), ('1212013675', '(추천) 철사밴드_40mm__(통(100EA))/ 호스밴드 호스조임 조임밴드 조임쇠 파이프밴드 철사밴드 철밴드조임'), ('1218797115', '[핫트랙스] Black 1타스 - [Shachihata] 일본 사찌하타-Artline 유성매직(유성마'), ('1648361433', 'i1 꽃을든남자 헤나 헤어젤(슈퍼홀드) 500ml/a'), ('1444013895', '동방애왕뜸기+진영감생명쑥봉(50개)+코끼리라이터'), ('1034144940', '1300K [투플레이스] 마르니 장스탠드'), ('1529816146', '김명희 우리애들밥상 고소감자 부침가루'), ('1588016425', 'HS_905_이지접톱270 절단기 톱날 접이식톱 접톱 절삭'), ('1077050737', '[ETUDEHOUSE]에뛰드하우스 내 피부를 부탁해 로얄젤리 마스크20ml_1매'), ('1533925346', 'GOGO!테이스티

In [12]:
noun_train, noun_test, yy_train, yy_test=train_test_split(nouns_list,y_list,test_size=0.2, random_state=42)
print (noun_train)
#print(noun_test)
#print (yy_train)

[('1284373589', '도그 도그차일드 차일드 레오파드 레오파드애견신발 애견 신발 노랑 강아지 강아지슈즈 슈즈 애 '), ('1603361243', '불스 방청 방청윤활제 윤활제 윤활 윤활방청제 방청제 방청유 유 윤활유 녹 '), ('1086777272', '쿠스쿠 쿠스쿠스파이 스파이 블루 베리 치즈 치즈파이 파이 8조각 조각 '), ('1367029764', '파 파사바체 사바 체 유리 계량컵 주방 주방용품 용품 계량 컵 조리 조리도구 도구 계량용품 '), ('1687461732', '제주 코 코코몽에코 몽 에코 파크 이즈 이즈랜드 랜드 제주 제주도 도 관광지 2곳 곳 패키지 입장권 할인 기 기프트제주 프트 승마 승마체험 체험 제주승마할인 승마장 장 추천 가격 가격말 말 말타기 '), ('1639169754', '발목 서포트 압박 압박밴드 밴드 발목보호대 보호대 무 '), ('1470415954', '중고 서버 '), ('1255743159', '투 투핫 핫 로 로키티 키 티 킹 킹컵 컵 세트 '), ('1611544124', '인테리어 스위치 스위치커버 커버 그래픽 그래픽스티커 스티커 파인애플 '), ('1486501072', '멋 캐주얼 정장 스펜더 멜빵 '), ('1549175351', '미 미코아이 코 아이 티 아트 아트조이 조이 명화 명화그리기 그리기 숲 숲속 속 작은집 '), ('1401228966', '프라이버시 필터 '), ('1530257201', '인도 사우디 사우디아라비아 아라비아 등 해외여행 답 '), ('1321872342', '스포츠 고무 고무연식 연식 야구공 12개입 개입 '), ('1578581418', '다람쥐 고양이 고양이장난감 장난감 고양 고양이용장난감 이용 캣 캣장난감 '), ('1036567080', '무료 무료배송 배송 랄 랄라룹시 라 룹 시 윈 윈터 드레스 터 패션 패션세트 세트 인형 매 윈터드레스 인형옷 옷 인 '), ('1651852477', '보브 듀얼 커버 크림 21호 호 라이트 톤 톤업 업 '), ('

In [13]:


#print(hi)
#print (my_train)
trans=vectorizer.transform(map(lambda i:i[1],noun_train))
print(trans)

  (0, 9587)	1
  (0, 9623)	1
  (0, 12022)	1
  (0, 13046)	1
  (0, 13048)	1
  (0, 14067)	1
  (0, 14068)	1
  (0, 20197)	1
  (0, 21073)	1
  (0, 22280)	1
  (1, 16809)	1
  (1, 16810)	1
  (1, 17846)	1
  (1, 25196)	1
  (1, 25198)	1
  (1, 25200)	1
  (2, 3664)	1
  (2, 17127)	1
  (2, 18011)	1
  (2, 27006)	1
  (2, 28477)	1
  (2, 29413)	1
  (3, 10101)	1
  (3, 10105)	1
  (3, 10106)	1
  :	:
  (6797, 28303)	1
  (6797, 30846)	1
  (6797, 32205)	1
  (6798, 9329)	1
  (6798, 9343)	1
  (6798, 11613)	1
  (6798, 17757)	1
  (6798, 21116)	1
  (6798, 28881)	1
  (6798, 28884)	1
  (6798, 33276)	1
  (6798, 33422)	1
  (6799, 1052)	1
  (6799, 2692)	1
  (6799, 14223)	1
  (6799, 14718)	1
  (6799, 16835)	1
  (6799, 16839)	1
  (6799, 21551)	1
  (6799, 22743)	1
  (6799, 25044)	1
  (6799, 25865)	1
  (6799, 29732)	1
  (6799, 31320)	1
  (6799, 31865)	1


In [14]:

for c in [1,5,10]:
    clf = LinearSVC(C=c)
    clf.fit(trans, yy_train)
    print (c,clf.score(vectorizer.transform(map(lambda i:i[1],noun_test)), yy_test))

1 0.642352941176
5 0.609411764706
10 0.599411764706


In [15]:
tmp_list=list(map(lambda i : i[1],eval_x_text_list))
print(tmp_list)

['acchousee 스프레이/기아/오피러스 /2006-2007/Z8/라이트메탈/주문제품(15일예상)', '[엑스포넷주식회사]코일탭_M162.0_18.6mm_1EA 절삭공구 작업공구 공구류 코일탭 탭 공구 빠른배송', '[오가닉맘]펭코남아목도리(MBW7MF02) (인천점)', 'TASCO INFOCUS 골프거리측정기 골프거리 측정 망원경', '(KNF)극동고성능해빙기/고성능전기해빙기/10kw', '(P)인두 플라이어 드라이버등 공구세트 수공구 공구', '[토이트론]실바니안패밀리-유치원학예회세트(3589)', '[누리]사이즈가 크고 튼튼한 뉴테크 좌식 바베큐그릴', '국내산 혀클리너 x 1개(낱개)혀 크리너/입냄새 제거/구강 청결제/여행용/혀간 칫솔 세트/치실/치약/백태', '(추천) 철사밴드_40mm__(통(100EA))/ 호스밴드 호스조임 조임밴드 조임쇠 파이프밴드 철사밴드 철밴드조임', '[핫트랙스] Black 1타스 - [Shachihata] 일본 사찌하타-Artline 유성매직(유성마', 'i1 꽃을든남자 헤나 헤어젤(슈퍼홀드) 500ml/a', '동방애왕뜸기+진영감생명쑥봉(50개)+코끼리라이터', '1300K [투플레이스] 마르니 장스탠드', '김명희 우리애들밥상 고소감자 부침가루', 'HS_905_이지접톱270 절단기 톱날 접이식톱 접톱 절삭', '[ETUDEHOUSE]에뛰드하우스 내 피부를 부탁해 로얄젤리 마스크20ml_1매', 'GOGO!테이스티 우유먼치 (25p)', '1p 지구 스펀지볼(10cm) 미니공 스폰지공 미니볼 스폰지볼 스펀지공 스펀지볼', '[베어파우](BEARPAW) GODETIA NAVY(womens) K460015GD-W', 'H]점멸식 LED 네온/블랙보드 BE127/홍보/메뉴판/광고', 'No350/AK-47에어건/BB탄총/비비탄총', 'S자 스틸 푸쉬업바/요가학원판매용 헬스용품점판매용', '(MST시원한 가야 알로에농장(1.5리터)x12페트 알로에음료 음료수 알로에주스 음료 과일음료', '[GRET](홍콩)홍콩 디즈니랜드

In [16]:
#print(eval_x_text_list)

test1=[]
for i in range(0,len(tmp_list)):
    new_list=kkma.nouns(tmp_list[i])
    no_num_data=[]
    tt=""
    for j in new_list:
        if isNumber(j) is False :
            for k in range(0,len(j)):
                tt+=j[k]
            tt+=' '
    test1.append((eval_x_text_list[i][0],tt))
print(test1)




[('720988653', '스프레이 기아 오피 오피러스 러스 라이트 라이트메탈 메탈 주문 주문제품 제품 15일예상 일 예상 '), ('1378489203', '엑스포 엑스포넷주식회사 넷 주식 주식회사 회사 코일 코일탭 탭 절삭 절삭공구 공구 작업 작업공구 배송 '), ('1621140511', '오가 오가닉맘 닉 맘 펭 펭코 코 목도리 인천 인천점 점 '), ('1549548735', '골프 골프거리측정기 거리 측정기 골프거리 측정 망원경 '), ('1642215271', '극 극동고성능해빙기 동고 성능 해빙기 고성능 고성능전기해빙기 전기 '), ('1594181530', '인두 플라이 드라이버 드라이버등 등 공구 공구세트 세트 수공구 '), ('1002465039', '토이 토이트론 트론 실 실바 바 안 안패밀리 패밀리 유치원 유치원학예회세트 학예회 세트 '), ('1592587497', '누 누리 리 사이즈 뉴 뉴테크 테크 좌식 바베큐 '), ('181355585', '국내산 혀 혀클리너 클리너 1개 개 낱개 크리너 입 입냄새 냄새 제거 구강 청결제 여행용 혀간 간 칫솔 세트 치 치실 실 치약 백태 '), ('1212013675', '추천 철사 철사밴드 밴드 통 호스 호스밴드 호스조임 조 임 조임쇠 파이프 파이프밴드 철 철밴드 '), ('1218797115', '트랙스 1타스 타스 일본 사 사찌하타 찌 하타 유성 유성매직 매직 유성마 마 '), ('1648361433', '꽃 남자 헤어 헤어젤 젤 슈퍼 슈퍼홀드 홀드 '), ('1444013895', '동방 동방애왕뜸 애 왕 뜸 기 진 진영감생명쑥 영감 생명 쑥 봉 50개 개 코끼리 코끼리라이터 라이터 '), ('1034144940', '투 투플레이스 플레이스 장 장스탠드 스탠드 '), ('1529816146', '김 김명희 명희 우리 애 밥상 고소 고소감자 감자 부침 부침가루 가루 '), ('1588016425', '접톱 접톱270 절단기 톱날 접이 접이식톱 식 톱 절삭 '), ('1077050737',

In [17]:
pred_list = clf.predict(vectorizer.transform(map(lambda i:i[1],test1)))
print(pred_list)

[0 1 2 ..., 3 7 3]


In [ ]:
import requests
name='김동현(97)'
nickname='Sword_Master'
mode='test'
param = {'pred_list':",".join(map(lambda i : str(int(i)),pred_list.tolist())),
         'name':name,'nickname':nickname,'mode':mode}
d = requests.post('http://eval.buzzni.net:20001/eval',data=param)

print (d.json())


In [ ]:
### cnn

In [94]:
pid_img_feature_dict = {}
with open("refined_category_dataset.img_feature.dat") as fin:
    for idx,line in enumerate(fin):
#        if idx%100 == 0:
#            print(idx)
        pid, img_feature_str = line.strip().split(" ")
        img_feature = (np.asarray(list(map(lambda i : float(i),img_feature_str.split(",")))))
        pid_img_feature_dict[pid] = img_feature
#        print (line)
#        break
        

In [95]:
from scipy import sparse 

In [96]:
img_feature_list = []
for pid, name in noun_train:
    print(pid, name)
    if pid in pid_img_feature_dict:
        img_feature_list.append(pid_img_feature_dict[pid])
#print (len(pid_img_feature_dict[pid]),type(pid_img_feature_dict[pid]))
#         break
    else:
        img_feature_list.append(np.zeros(1000))
#     break

1284373589 도그 도그차일드 차일드 레오파드 레오파드애견신발 애견 신발 노랑 강아지 강아지슈즈 슈즈 애 
1603361243 불스 방청 방청윤활제 윤활제 윤활 윤활방청제 방청제 방청유 유 윤활유 녹 
1086777272 쿠스쿠 쿠스쿠스파이 스파이 블루 베리 치즈 치즈파이 파이 8조각 조각 
1367029764 파 파사바체 사바 체 유리 계량컵 주방 주방용품 용품 계량 컵 조리 조리도구 도구 계량용품 
1687461732 제주 코 코코몽에코 몽 에코 파크 이즈 이즈랜드 랜드 제주 제주도 도 관광지 2곳 곳 패키지 입장권 할인 기 기프트제주 프트 승마 승마체험 체험 제주승마할인 승마장 장 추천 가격 가격말 말 말타기 
1639169754 발목 서포트 압박 압박밴드 밴드 발목보호대 보호대 무 
1470415954 중고 서버 
1255743159 투 투핫 핫 로 로키티 키 티 킹 킹컵 컵 세트 
1611544124 인테리어 스위치 스위치커버 커버 그래픽 그래픽스티커 스티커 파인애플 
1486501072 멋 캐주얼 정장 스펜더 멜빵 
1549175351 미 미코아이 코 아이 티 아트 아트조이 조이 명화 명화그리기 그리기 숲 숲속 속 작은집 
1401228966 프라이버시 필터 
1530257201 인도 사우디 사우디아라비아 아라비아 등 해외여행 답 
1321872342 스포츠 고무 고무연식 연식 야구공 12개입 개입 
1578581418 다람쥐 고양이 고양이장난감 장난감 고양 고양이용장난감 이용 캣 캣장난감 
1036567080 무료 무료배송 배송 랄 랄라룹시 라 룹 시 윈 윈터 드레스 터 패션 패션세트 세트 인형 매 윈터드레스 인형옷 옷 인 
1651852477 보브 듀얼 커버 크림 21호 호 라이트 톤 톤업 업 
1629587718 버터링 버터링쿠키 쿠키 요 요쿠르트 쿠르트 국내산 국내산쿠키 2개 개 강아지 강아지과자 과자 
1589969317 태성 캐논 캐논정품후드 정품 후드 전용 
1624338116 가을 날라리 단가 단가라 라 상하 
1630907693

1393521133 눈사람 옐로 옐로우 우 크리스마스 크리스마스장식 장식 테이블 테이블데코 데 코 파티 파티장식 트리 트리데코 장식인형 인형 
1190284866 버드 골키퍼 장갑 모음 축구 축구장갑 
102430123 굿 굿프랜드 프 랜드 캔 닭 닭고기 고기 야채 애견 애견캔 
1613677753 펜더 펜더스콰 스콰 베이스 베이스기타 기타 엠 엠프 프 
1576061071 압소 압소바 바 레이 60수 수 오가 오가닉 닉 신 신생모 생모 
1435442997 말레이시아 조 조호바 호 바 호텔 호텔예약 예약 바 바유 유 마리나 리조트 호텔검색 검색 호텔가격 가격 
1596881546 오 오가게 가게 피 피턴진걸 턴 진 걸 여 기모 하이 밑단 기모반 밑단컷팅블랙 반 컷 팅 블랙 데님 데님스키 스키 
1312832315 아림 아림전기 전기 세라믹 인두 인두기 기 일자 일자형 형 납땜 납땜기 납땜인두 전기인두 전기인두기 인두공구 공구 용접 
1297563090 무료 특가 색 색동 동 앞치마 다용도 다용도앞치마 주방 주방용품 용품 주방소품 소품 주방소모품 소모품 
1669597985 미니 미니언즈 언 즈 삼선 삼선슬리퍼 슬리퍼 민트 미니언즈슬리퍼 
1440234501 ㅁ ㅁ시ㅁ 시 공 유무선 ㅁ일ㅁ 공유기ㅁ당 일 유기 당 발 발ㅁ송ㅁ 송 
1571368021 바보 바보친구 친구 멍 간식 고구 고구마사사미400 마사 사미 
1116405616 크리스탈 보계 디지털 보기 다이어트 다이어트용품 용품 헬스 헬스용품 
1436391203 쿠쿠 쿠쿠토이즈 토 이즈 아기 아기곰 곰 놀이방 레드 놀이 방 볼 볼풀장 풀장 완구 
1575967677 해외 
1630912237 신 신개념 개념 무지 무지실크벽지 실크 벽지 심플 플레인 바이올렛 1롤 롤 5평 평 
1219081581 신한 신한튜브낱색 튜브 낱 색 호분 동양화 채색 물감 서예 서예용품서예도구 용품 도구 화선지 서예붓 붓 낙관 서예재료 재료 먹 먹물 
1518010306 라 라쳇주먹드라이버 쳇 주먹 드라이버 
148759256

1620919367 게르마늄 게르마늄성분 성분 함유 이슬 깻잎 깻잎2 1박스 박스 
1486753280 현대 현대백화점 백화점 자연 블루 블루밍 밍 리프팅 전 
1599986298 요 요넥스 넥스 스포츠 배드민턴 여성 긴팔 긴팔티 티 
1548170649 테이프 테이핑 근육 근육테이프 근육테이핑 근육면테 면 테 
1233889211 출퇴근 출퇴근기록기카트리지 기록기 카트리지 전용 
1614017142 주니어 드럼 드럼세트 세트 
1564461690 네트워크 네트워크분배기 분배기 고 고해상도 해상도 모니터 자동 분 
1558474121 천연 천연영양새사료 영양 새 사료 새사료 앵무새 앵무새사료 
1559808152 밀가루 피자 피자전용 전용 백설 분말 
1605746300 세신 펜치 찌 팬 니빠 공구 수공구 바이스 
1436915820 스프레이 극세사 밀대 청소기 사각 360도회전 도 회전 부착 
1673484618 넥스트 스파크 장기 장기렌트 렌트 전화 전화상담문 상담 문 환영 
1591276779 기가 기가바이트 바이트 윈드 윈드포스 포스 
1625862078 아디다스 리니어 후드 
411496011 롯데 롯데칠성 칠성 옥 옥수수수염차 수수 수염 차 30캔 캔 
1489790054 추천 오렌지 오렌지락 락 사각 3ㅡ2 ㅡ 스테인레스 사각밀폐용기 밀폐 밀폐용기 용기 냉장고 냉장고밀폐용기 밀 밀페용기 페 밀패용기 패 스텐 스텐밀폐 
1532065360 영 영공방 공방 모던 모던하우스 하우스 
1368229349 디 아파트 아파트먼트 먼트 호텔 비 
1380285122 나노 나노전자 전자 전기 전기돈풍기 돈 풍기 난방 난방면적 면적 ㎡ 튜브 튜브형 형 히터 발열량 10,320 
1667724716 옵티 옵티카 카 포로 포로501752쌍안경 쌍안경 낚시 낚시용품 용품 망원경 
1135960631 리 리키안 키 안 고 고기능성 기능성 기모 융 방한 9부타이즈 부 타이즈 
1367204474 피치 피치픽스 픽스 얼 얼라인미 라인 미 이너 필드 필드용품 용품 골프 골프용

1491031476 스카 스카샵 샵 캠핑 캠핑캐리 캐리 더블 더블도어 도어 오픈 오픈형 형 애견 애견캐리어 캐리어 고양이 고양이이동장 이동 장 더블도어애견캐리어 애견이동 이동 
1678302337 이스턴 파크 강원 강원리조트 리조트 평창 평창군 평창군리조트 군 강원리조트추천 추천 평창군리조트추천 강원여행 여행 평창군여행 
1433752649 경첩 정첩 문 방문 도 가구 
1529716184 카드 카드10 일본 간사이 호 패스 성인 아동 7일 일 
1519419913 프로 프로베스트 베스트 퍼피 프로베스트사료 사료 애견 애견사료 강아지 강아지사료 
1595850999 낫소 스타 족 족구화 구화 레 레독스 독스 라이트 실버 족구 신발 운동화 
1566347828 바보 바보사랑 사랑 에이 에이라이크 라이크 방 방수 수 전자 전자시계 시계 
1591628788 용수 용수공업 용수공업사 공업 사 고속 고속절단기 절단기 원형 원형톱 톱 원형톱날 톱날 
1612875712 별가 쌀 쌀떡볶이1.5 떡볶이 10개 개 1박스 박스 
1562080358 뽕잎 뽕잎가루 가루 
1659777589 마레스 네모 이드 전용 전용보호커버 보호 커버 
1654094062 무료 무료배송 배송 웅진 음료수 하늘 하늘보리 보리 12개 개 
1492173965 쏘로 쏘로사떼 사 떼 노 레이스 내장 내장캡 캡 스포츠 컵 컵브라-3가지 브라 가지 색상 여성 여성속옷 속옷 브래지어 고급 고급브라 
1149001257 싹 휴대용 수동 안마기 
1665415413 스톰 다운 다운스프링 스프링 벨 벨로스터 로스터 
1454947010 알뜰 알뜰쇼핑 쇼핑 정품 흑백 레이저 레이저프린터 프린터 토너 2.000매 매 잉크 잉크카트리지 카트리지 정품잉크 레이 
1185854979 알루미늄 알루미늄셀카봉 셀 카 봉 니콘 니콘호환리모콘 호환 리모콘 세트 세트상품 상품 셀카포드 포드 모노 모노포드 삼각대 미니 접사 
1274409784 유기농 토마토 토마토즙 즙 30봉 봉 산지 산지직배송 직 배송 
510658245 설

1325506056 악기 악기대통령 대통령 던롭 핑거 핑거피크 피크 화이트 크기별 선택 선택구매 구매 
1634272721 5개 개 적 적끈 끈 대 약 약250 에어 에어캡 캡 포장 포장용품 용품 포장끈 
1529350397 동서 동서가구 가구 파 파베드-2인 베드 인 
1012640632 마 리 스타 청진기 신생아 독일 독일완제품 완제품 품격 내 모습 비교 
1633472905 꽃 남자 케라틴 헤어 헤어젤500 젤 컷트 웨이브 
1626002594 방한 방한마스크 마스크 체크 추운 겨울 대비 순면 순면마스크 
1537323208 랜 랜마트 마트 공 공유기 유기 케이블 인터넷 인터넷와이파이케이블 와이 파이 인터넷케이블 공유 공유기와이파 기와 이파 
1116209074 재단기 1회 회 재단 30매 매 로터리 로터리형 형 대량 문서 문서고 문서고정대 정대 펠로우즈 
1505411061 호주 골드 코스트 호텔 호텔예약 예약 오스카 온 메인 리조트 호텔검색 검색 호텔가격 가격 
1348213798 토이 토이트론 트론 사운드 퓨처 퓨처북 북 코코몽 명작 명작동화 동화 
943047925 아이리버 딕플 전용 케이블 데이터 데이터전송 전송 충전 노이즈 노이즈필터장착 필터 장착 초고 초고속배송 속 배송 
1401072079 러스트 럼 녹 녹방지 방지 페인트 스프레이 메탈 메탈릭 릭 
1273272171 호 닷컴 캐주얼 캐주얼점퍼 점퍼 애 블루 블루종 종 
1493223731 휴 휴롬 롬 원액 드럼 드럼용량 용량 저속 저속착즙방식 착즙 방식 간이 간이세척 세척 가능 주스 주스캡 캡 
1507749088 계양 레이 레벨 충전 툴 툴마트 마트 
1671204995 애견 애견미용 미용 숱 숱가위 가위 강 강아지털손질 아지 털 손질 
1033977163 출산 출산필수세트 필수 세트 순면 순면사각기저귀5 사각 기저귀 매 순면손수건20매 손수건 
1095547732 래피드 전자 전자체온계 체온계 디지털 
1578337861 바 랜드 뽀로 덤프 덤프트럭 트럭 장난감 자동차 
1118929

1520139805 프로 프로모릭스 모 릭스 무릎 무릎보호대 보호대 
1234004512 환자 침대 병원 병원침대 환자침대 
588108288 판매 판매중 중 타올 키친 키친타올 순수 순수펄프사용 펄프 사용 토 토미키친타올 미 엠보싱 가정용 집들이 선물 핸드 핸드롤 롤 
1359688023 로지 레드 코디 2인용베개 인용 베개 솜 솜포함 포함 40수 수 순면 블루 블루밍데코 밍 데 코 
1632189876 웅진 자연은 알로에 주스 과즙 
1598324607 멀티 액션 무릎 스트랩 보호대 스포츠 
821469247 아이 아이수 수 국내산 고창 복분자 진액 4병 병 복분자원액 원액 복분자즙 즙 복분자진액 복분자엑기스 엑기스 복분자쥬스 쥬스 
1637357531 홈 홈아트 아트 악 악어구구헤라 어구 구 헤라 대 신발 신발주걱 주걱 휴대용 휴대용구두 구두 
1501310703 고압 솔레노이드 밸브 정수기 정수기부품 부품 
1577582861 꼬꼬마 큐빅 하트 이름표 목걸이 핑크 
1646077583 오토 오토첵 첵 혈당 혈당측정지 측 정지 스트립 
1401922742 영운 영운악기 악기 드럼 드럼스토어 스토어 콩가 콩가세트 세트 더블 더블스탠드포함 스탠드 포함 빈티지 버스트 
1642316502 렉슨 금속 금속절단기 절단기 톱날 톱날별도 별도 밴 밴드쏘 드쏘 전동 전동공구 공구 
1483333340 무선 무선네트웍 네트웍 블루투스 리시버 화이트 
1143724782 서울특별 서울특별시 시 구로 구로구 구 위성 위성관내 관내 서울시 지도 고급 고급양면코팅 양면 코팅 
1513965810 마 케어 베이 베이비여아핑크 비 여아 핑크 타이즈 
809304699 당일 동해 동해바다 바다 생물 홍 홍게 게 유 대게 
1311132653 무선 무선마이크2대 마이크 대 휴대용 무선앰프 앰프 스피커 
1595199311 하이닉스 노트북 
1443291679 스카 스카샵 샵 캐주얼 크로스 크로스백 백 백 여성 여성크로스백 캐주얼미니백 미니 패션 패션미니백 패션크로스가방 가방 상품 상품

1632854273 바보 바보친구 친구 주 기본 슬랙스 
1638384507 엔프라 리 리킷 킷 래디언스 비 비비크림 크림 
1568292441 매 매티파 티 파 프라이 
1312219842 퓨 미네랄 립 립앤아이 앤 아이 메이크업 리무버 스윗 스윗로즈 로즈 
1509435822 오너 볼락 소프트 소프트베이트 베이트 소프트웜 웜 루어 루어낚시웜 낚시 
1666015755 파 알 카바레 쇼 
1509183201 텍스 접이 접이식침대 식 침대 열 열매트 매트 이동식 이동식침대 기능성 기능성침대 간이 간이침대 
1554236079 약초 두충 국내산 
1464263341 소피아 건강 팔찌 음이온 게르마늄 
1681889941 톰 톰보 보 조각도 세트 6본조 본조 판화 판화용품 용품 목공 공구 조소 
1650866649 스탠 스탠물병 물병 약 약740 블루 강아지 강아지용품 용품 애견 애견용품 
1659587314 61건반 건반 키보드 키보드케이스 케이스 
1579021086 요즘 난리 프로 프로유 유 차 차콜석고 콜 석고 25개 개 모델 
1558029597 어린이 원목 원목옷걸이 옷걸이 아동 아동옷걸이 어린이옷걸이 
1038783021 추천 미니 미니팡 팡 다기능 다기능안마기 안마기 어깨 어깨안마기 진동 진동안마기 다리 다리안마기 목 목안마기 핸드 핸드안마기 목어깨안마 안마 
1138499412 싱크대 배수구 속 속뚜껑 뚜껑 거름망 대형 전용 마개 싱크대속 씽크대 씽크대뚜껑 
1444448641 하성 하성전자 전자 펫 클리퍼 애견 애견바리깡 바리 깡 
1521197412 트라이 트라이던트 던트 16형 형 선풍기 스탠드 스탠드형 
1596980288 젖병 젖병세트 세트 로도 로도기 기 강아지 강아지젖병 애견 애견젖병 강 
1664933944 미 미쟝 쟝 스타일 스타일케어 케어 프로페셔널 스트롱 홀드 헤어 
1620867062 스파이 거미줄 방한 방한부츠 부츠 유아 아동 아동부츠 
1392417517 폭스바겐 항균 항균필터 필터 활성탄 1998년 년 2005년 에어컨 

1096876890 그루 그루머스 머스 통 통타입 타입 윙 윙사사미 사사 미 지 지펫 펫 애견 애견용품 용품 
1602767982 바보 바보친구 친구 아쿠아 아쿠아리스트 리스트 페리하 고급 고급형 형 히터 자동 전자 전자히 히 
1429902064 크리스찬 락 락토필듀오 토필 듀오 30캡슐 캡슐 유산균 여성 
1699487539 제주 본태 본태박물관 박물관 다이나 다이나믹 믹 이즈 제주 제주도 도 관광지 2곳 곳 패키지 입장권 할인 기 기프트제주 프트 승마 승마체험 체험 제주승마할인 승마장 장 추천 가격 가격말 말 말타기 
1447823683 의료 핀셋 가위 실습 수술 의료기기 기기 
1289851795 사슴 사슴벌레 벌레 암컷 미만 
1444650016 인증 인증점 점 유무선 유무선공유기 공 유기 와이 와이파이 파이 인터넷 
1431660554 금 금홍팬시 홍 팬시 겨울 겨울왕국 왕국 핸드백 색칠 색칠놀이 놀이 1개 개 랜덤 랜덤발송 발송 색칠북 북 색칠공부 공부 스티 
1549174107 미 미코아이 코 아이 티 아트 아트조이 조이 명화 명화그리기 그리기 황도 황도12궁 궁 
1464294973 피 피씨사자 씨 사자 배송 추첨 추첨응모 응모 증정 
1521171673 3월 월 특가 한화 넷 일산 입장권 아쿠아 정글 스카이 곳 나들이 24일 일 4월 16일 일 사용 가능 
1584174616 시 맨투맨 긴팔 긴팔티셔츠 티셔츠 
1526020197 반테린코 서포터 팔꿈치 탄력 탄력밴드 밴드 팔꿈치보호대 보호대 
1643053374 바보 바보사랑 사랑 아이 아이디스타 디 스타 유스 유스테어 테어 화이트 브릭 스탠드 
1511797590 무료 무료배송 배송 발 발펌프 펌프 
1165413496 정품 곳 등 인트 인트플라이 플라이 정비 정비공구 공구 자동차 자동차공구 인 동차 동차공구 차량 
1669542719 제도 제도샤프 샤프 검정 회사 회사납품 납품 
1501363568 빛 진주 진주체인 체인 
1661295787 댕기 댕기머리 머리 해운 샴푸 
1560457

1507768139 문 문펫 펫 벨 자동 자동핀 핀 5색 색 4개 개 20개 1판 판 리본 애견 애견악세서리 악세 악세서리 서리 애견용품 용품 
1607439546 아프리카 프리미엄 젬 젬베 베 프리미엄급 급 악 
1472281615 브리츠 브리츠인터내셔널 인터내셔널 사운드 스피커 
1534025361 무료 무료배송 배송 제우스 옴 옴므 므 엑스 엑스라인 라인 샌들 2색상 색상 
166900169 최신형 충전 충전가능 가능 멀티 멀티플러그 플러그 해외여행 해외여행필수품 필수품 답 답터 터 세계 어디 사용 최저가 최저가판매 판매 
1485480438 에프에이 올 올스왑 스왑 100매 매 5박스 박스 500매 알콜 알콜솜 솜 
1708916080 카이 스파 진주 진주마사지 마사지 
1500389884 포보스 포보스골프정품 골프 정품 항공 항공커버 커버 
1244060040 롯데 이 이라이프 라이프 휴대용 휴대용가스렌지 가스 가스렌지 렌지 고화 대 누리 생활 생활용품 용품 버너 캠핑 캠핑용품 가스레인지 레인지 낚시 낚시용품 부탄 부탄가스 
1134141497 기타 기타케이스 케이스 스탠다드 어쿠스틱 어쿠스틱기타 가방 가벼운 가벼운소재 소재 
1633131121 엔프라 데이 데이시스 시스 로얄 로얄비 비 셜 필링 필링젤 젤 
1368187645 선물 선물가게 가게 격자 격자무늬 무늬 알파벳 무릎 무릎담요 담요 무 
1392615812 2개국 개국 보기 이태 이태리 리 스위스 일주 89일 일 
1514123202 애견 애견가방 가방 모음 강아지 강아지가방 고양이 애완 캐리어 강아지용품 용품 이동 이동장 장 강아지집 집 
1320354868 브리더 진도 진도장 장 중 애견 애견철장 철장 
1196538793 야간 야간응원용 응원 용 야광 팔찌 무료 무료인쇄 인쇄 인쇄판촉물전문 판촉물 전문 소량 소량상담가능 상담 가능 야광팔찌 야광봉 봉 야광밴드 밴드 
1708912015 상해 데이 데이투어 투어 
1236220633 드림 드림베이비 베이비 자외선 자외선차단 차단 유모차 

In [97]:
img_feature_test_list = []
for pid, name in noun_test:
    if pid in pid_img_feature_dict:
        img_feature_test_list.append(pid_img_feature_dict[pid])
    else:
        img_feature_test_list.append(np.zeros(1000))

In [98]:
print(len(img_feature_list))
print(len(img_feature_test_list))

6800
1700


In [99]:

hi=vectorizer.transform(map(lambda i : i[1],noun_train))
bye=vectorizer.transform(map(lambda i : i[1],noun_test))
concat_x_list = sparse.hstack((hi, img_feature_list),format='csr')
concat_test_x_list = sparse.hstack((bye, img_feature_test_list),format='csr')


In [100]:

for c in [0.2]:
    clf2 = LinearSVC(C=c)
    clf2.fit(concat_x_list, yy_train)
    print (c,clf2.score(concat_test_x_list, yy_test))


0.2 0.686470588235


In [101]:
del pid_img_feature_dict

In [102]:
#print(noun_test)

In [103]:
pid_img_feature_dict = {}
with open("refined_category_dataset.img_feature.eval.dat") as fin:
    for idx,line in enumerate(fin):
#        if idx%100 == 0:
#            print(idx)
        pid, img_feature_str = line.strip().split(" ")
        img_feature = (np.asarray(list(map(lambda i : float(i),img_feature_str.split(",")))))
        pid_img_feature_dict[pid] = img_feature
#         print (line)
#         break
        

In [108]:
test_x_text_list = []
with open("soma8_test_data.dat",encoding=enc) as fin:
    for line in fin.readlines():
        info = json.loads(line.strip())
        test_x_text_list.append((info['pid'],info['name']))

#pred_list = clf.predict(vectorizer.transform(map(lambda i : i[1],eval_x_text_list)))

In [109]:
img_feature_eval_list = []

In [110]:
for pid, name in test_x_text_list:
    if pid in pid_img_feature_dict:
        img_feature_eval_list.append(pid_img_feature_dict[pid])
    else:
        img_feature_eval_list.append(np.zeros(1000))


In [111]:
print (len(img_feature_eval_list), len(eval_x_text_list))

1292 1292


In [112]:
print(eval_x_text_list[0])
#print(img_feature_eval_list[0])
print(test_x_text_list[0])
#print(img_feature_test_list[0])
print(noun_test[0])

('720988653', 'acchousee 스프레이/기아/오피러스 /2006-2007/Z8/라이트메탈/주문제품(15일예상)')
('720988653', 'acchousee 스프레이/기아/오피러스 /2006-2007/Z8/라이트메탈/주문제품(15일예상)')
('1494008925', '백색 색 센스 에나멜 페인트 목재 철재 조색 ')


In [113]:
tmp_list=list(map(lambda i : i[1],eval_x_text_list))
print(tmp_list)

['acchousee 스프레이/기아/오피러스 /2006-2007/Z8/라이트메탈/주문제품(15일예상)', '[엑스포넷주식회사]코일탭_M162.0_18.6mm_1EA 절삭공구 작업공구 공구류 코일탭 탭 공구 빠른배송', '[오가닉맘]펭코남아목도리(MBW7MF02) (인천점)', 'TASCO INFOCUS 골프거리측정기 골프거리 측정 망원경', '(KNF)극동고성능해빙기/고성능전기해빙기/10kw', '(P)인두 플라이어 드라이버등 공구세트 수공구 공구', '[토이트론]실바니안패밀리-유치원학예회세트(3589)', '[누리]사이즈가 크고 튼튼한 뉴테크 좌식 바베큐그릴', '국내산 혀클리너 x 1개(낱개)혀 크리너/입냄새 제거/구강 청결제/여행용/혀간 칫솔 세트/치실/치약/백태', '(추천) 철사밴드_40mm__(통(100EA))/ 호스밴드 호스조임 조임밴드 조임쇠 파이프밴드 철사밴드 철밴드조임', '[핫트랙스] Black 1타스 - [Shachihata] 일본 사찌하타-Artline 유성매직(유성마', 'i1 꽃을든남자 헤나 헤어젤(슈퍼홀드) 500ml/a', '동방애왕뜸기+진영감생명쑥봉(50개)+코끼리라이터', '1300K [투플레이스] 마르니 장스탠드', '김명희 우리애들밥상 고소감자 부침가루', 'HS_905_이지접톱270 절단기 톱날 접이식톱 접톱 절삭', '[ETUDEHOUSE]에뛰드하우스 내 피부를 부탁해 로얄젤리 마스크20ml_1매', 'GOGO!테이스티 우유먼치 (25p)', '1p 지구 스펀지볼(10cm) 미니공 스폰지공 미니볼 스폰지볼 스펀지공 스펀지볼', '[베어파우](BEARPAW) GODETIA NAVY(womens) K460015GD-W', 'H]점멸식 LED 네온/블랙보드 BE127/홍보/메뉴판/광고', 'No350/AK-47에어건/BB탄총/비비탄총', 'S자 스틸 푸쉬업바/요가학원판매용 헬스용품점판매용', '(MST시원한 가야 알로에농장(1.5리터)x12페트 알로에음료 음료수 알로에주스 음료 과일음료', '[GRET](홍콩)홍콩 디즈니랜드

In [114]:
PPAP=[]
for i in range(0,len(tmp_list)):
    new_list=kkma.nouns(tmp_list[i])
    no_num_data=[]
    tt=""
    for j in new_list:
        if isNumber(j) is False :
            for k in range(0,len(j)):
                tt+=j[k]
            tt+=' '
    PPAP.append((eval_x_text_list[i][0],tt))
print(PPAP)


[('720988653', '스프레이 기아 오피 오피러스 러스 라이트 라이트메탈 메탈 주문 주문제품 제품 15일예상 일 예상 '), ('1378489203', '엑스포 엑스포넷주식회사 넷 주식 주식회사 회사 코일 코일탭 탭 절삭 절삭공구 공구 작업 작업공구 배송 '), ('1621140511', '오가 오가닉맘 닉 맘 펭 펭코 코 목도리 인천 인천점 점 '), ('1549548735', '골프 골프거리측정기 거리 측정기 골프거리 측정 망원경 '), ('1642215271', '극 극동고성능해빙기 동고 성능 해빙기 고성능 고성능전기해빙기 전기 '), ('1594181530', '인두 플라이 드라이버 드라이버등 등 공구 공구세트 세트 수공구 '), ('1002465039', '토이 토이트론 트론 실 실바 바 안 안패밀리 패밀리 유치원 유치원학예회세트 학예회 세트 '), ('1592587497', '누 누리 리 사이즈 뉴 뉴테크 테크 좌식 바베큐 '), ('181355585', '국내산 혀 혀클리너 클리너 1개 개 낱개 크리너 입 입냄새 냄새 제거 구강 청결제 여행용 혀간 간 칫솔 세트 치 치실 실 치약 백태 '), ('1212013675', '추천 철사 철사밴드 밴드 통 호스 호스밴드 호스조임 조 임 조임쇠 파이프 파이프밴드 철 철밴드 '), ('1218797115', '트랙스 1타스 타스 일본 사 사찌하타 찌 하타 유성 유성매직 매직 유성마 마 '), ('1648361433', '꽃 남자 헤어 헤어젤 젤 슈퍼 슈퍼홀드 홀드 '), ('1444013895', '동방 동방애왕뜸 애 왕 뜸 기 진 진영감생명쑥 영감 생명 쑥 봉 50개 개 코끼리 코끼리라이터 라이터 '), ('1034144940', '투 투플레이스 플레이스 장 장스탠드 스탠드 '), ('1529816146', '김 김명희 명희 우리 애 밥상 고소 고소감자 감자 부침 부침가루 가루 '), ('1588016425', '접톱 접톱270 절단기 톱날 접이 접이식톱 식 톱 절삭 '), ('1077050737',

In [115]:
x_feature_list = vectorizer.transform(map(lambda i : i[1],PPAP))

In [116]:
concat_test_x_list = sparse.hstack((x_feature_list, img_feature_eval_list),format='csr')


In [117]:
pred_list = clf2.predict(concat_test_x_list)

In [118]:

import requests
name='김동현(97)'
nickname='맞.왜.틀.'
mode='test'
param = {'pred_list':",".join(map(lambda i : str(int(i)),pred_list.tolist())),
         'name':name,'nickname':nickname,'mode':mode}
d = requests.post('http://eval.buzzni.net:20001/eval',data=param)
print (d.json())



{'msg': 'If you pull docker image before 2017-09-27 21:30,  pull your docker image again.', 'precision': 0.7352941176470589}
